In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

In [3]:
from tensorflow.keras.callbacks import TensorBoard

In [4]:
import pickle
import time

In [11]:
#NAME = 'Cats-vsdogs-cnn-64x2{}'.format(int(time.time())) # time stamp makes name unique if forget to change name
NAME = 'Cats-vsdogs-cnn-64x2-NO-DENSE{}'.format(int(time.time())) 

In [12]:
tensorboard = TensorBoard(log_dir='sgm_logs/{}'.format(NAME))
# to see tensorboard, type 'tensorboard --logdir sgm_logs' in dir that holds the logs dir  
# type \w into filter in tensorboard to see all graphs together

In [7]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [8]:
X = pickle.load(open("X.pickle",'rb'))
y = pickle.load(open('y.pickle','rb'))

In [9]:
X = X/255.0

In [13]:
model = Sequential()

#Conv 1
# 64 units, window size is 3 by 3
model.add(Conv2D(64, (3,3), input_shape = X.shape[1:])) # Conv accepts 2D dataset
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

# Conv 2
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())# Need to flatten since the final Dense layer wants 1D dataset

# Dense layer
#model.add(Dense(64))  # Remove dense layer for second attempt
#model.add(Activation('relu'))  # Remove dense layer for second attempt

#output layer
model.add(Dense(1))
model.add(Activation('sigmoid'))

#(This is not 10 layers)

model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

model.fit(X, y, batch_size=32, epochs=20, validation_split=0.1, callbacks=[tensorboard] )

Train on 22451 samples, validate on 2495 samples
Epoch 1/20
22451/22451 [==============================] - 97s 4ms/step - loss: 0.6134 - acc: 0.6574 - val_loss: 0.5831 - val_acc: 0.6862
Epoch 2/20
22451/22451 [==============================] - 96s 4ms/step - loss: 0.5156 - acc: 0.7469 - val_loss: 0.5120 - val_acc: 0.7527
Epoch 3/20
22451/22451 [==============================] - 104s 5ms/step - loss: 0.4774 - acc: 0.7733 - val_loss: 0.5307 - val_acc: 0.7399
Epoch 4/20
22451/22451 [==============================] - 102s 5ms/step - loss: 0.4560 - acc: 0.7861 - val_loss: 0.4716 - val_acc: 0.7727
Epoch 5/20
22451/22451 [==============================] - 98s 4ms/step - loss: 0.4334 - acc: 0.8009 - val_loss: 0.4704 - val_acc: 0.7768
Epoch 6/20
22451/22451 [==============================] - 98s 4ms/step - loss: 0.4123 - acc: 0.8126 - val_loss: 0.4622 - val_acc: 0.7780
Epoch 7/20
22451/22451 [==============================] - 94s 4ms/step - loss: 0.3946 - acc: 0.8217 - val_loss: 0.4511 - val_ac

## This is what Tensorboard looks like showing the network with (orange) and without (blue) the dense layer

<img src ='tensorboard-screengrab-2nets-compare.png'>

The loss on the test set when dense layer is removed is lower than the net with more layers, becoming optimal at ~12 epochs.  It looks like beyond 12 epochs, the network starts overfitting.  